In [1]:
import os

# Imports
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks, layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
print(tf. __version__)

2.18.0


In [4]:
image_size = 224
target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)
grid_shape = (1, image_size, image_size, 3)

batch_size = 16

In [5]:
train_dir = '/content/drive/MyDrive/Dataset_Split/train'

train_aug = ImageDataGenerator(
    # Rescale
    rescale=1/255.0,
    # Filling for W/H shift
    fill_mode="nearest",
    # Width and Height shift
    width_shift_range=0.2,
    height_shift_range=0.2,
    # Random zooms
    zoom_range=0.2,
    # Random Shearing aug
    shear_range=0.2,
)

# Read data from directory
train_data = train_aug.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical"
)

Found 24375 images belonging to 55 classes.


In [13]:
test_dir = '/content/drive/MyDrive/Dataset_Split/test'

# Augmentations for test data
test_aug = ImageDataGenerator(
    # Rescale
    rescale=1/255.0
)

# Read data from directory
test_data = test_aug.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical"
)

Found 6122 images belonging to 55 classes.


In [7]:
cats = list(train_data.class_indices.keys())

In [8]:
mbnet_v2 = keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=input_shape
)

# Stop from being trainable
mbnet_v2.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
inputs = keras.Input(shape=input_shape)

# Get the layer
x = mbnet_v2(inputs, training = False)

# Stack layers further
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(len(cats), activation="softmax")(x)

# Combine the model
model = Model(inputs=inputs, outputs=x)

# Summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 55)                  │          70,455 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,328,439 (8.88 MB)

 Trainable params: 70,455 (275.21 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [10]:
# Compile
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Define callbacks to use
early_stopping_cb = callbacks.EarlyStopping(monitor="loss", patience=3)

In [11]:
# Num epochs
epochs = 30

# Train model
history = model.fit(
    train_data,
    epochs=epochs,
    steps_per_epoch=150,
    callbacks=[early_stopping_cb]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 883s 6s/step - accuracy: 0.2482 - loss: 3.2677
Epoch 2/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 778s 5s/step - accuracy: 0.6656 - loss: 1.2397
Epoch 3/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 722s 5s/step - accuracy: 0.7211 - loss: 0.9826
Epoch 4/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 649s 4s/step - accuracy: 0.7623 - loss: 0.8017
Epoch 5/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 576s 4s/step - accuracy: 0.7900 - loss: 0.7058
Epoch 6/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 529s 4s/step - accuracy: 0.7955 - loss: 0.6726
Epoch 7/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 476s 3s/step - accuracy: 0.8006 - loss: 0.6436
Epoch 8/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 446s 3s/step - accuracy: 0.8257 - loss: 0.5746
Epoch 9/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 407s 3s/step - accuracy: 0.8243 - loss: 0.5628
Epoch 10/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 400s 3s/step - accuracy: 0.8112 - loss: 0.5770
Epoch 11/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 54s 347ms/step - accuracy: 0.8119 - loss: 0.6170
Epoch 12/30


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


150/150 ━━━━━━━━━━━━━━━━━━━━ 352s 2s/step - accuracy: 0.8308 - loss: 0.5353
Epoch 13/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 314s 2s/step - accuracy: 0.8414 - loss: 0.4841
Epoch 14/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 296s 2s/step - accuracy: 0.8779 - loss: 0.4253
Epoch 15/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 282s 2s/step - accuracy: 0.8358 - loss: 0.4977
Epoch 16/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8719 - loss: 0.4464
Epoch 17/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 243s 2s/step - accuracy: 0.8658 - loss: 0.4230
Epoch 18/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 234s 2s/step - accuracy: 0.8583 - loss: 0.4482
Epoch 19/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.8830 - loss: 0.3965
Epoch 20/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - accuracy: 0.8732 - loss: 0.4309
Epoch 21/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 206s 1s/step - accuracy: 0.8607 - loss: 0.4657
Epoch 22/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 29s 184ms/step - accuracy: 0.8604 - loss: 0.4304
Epoch 23/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 2

In [14]:
model.evaluate(test_data)

383/383 ━━━━━━━━━━━━━━━━━━━━ 1504s 4s/step - accuracy: 0.8591 - loss: 0.4415


[0.43333980441093445, 0.861973226070404]

In [15]:
model.save("Mobile_Net_trained.h5")

In [16]:
model.save('Mobile_Net_trained.keras')

In [ ]:
hist = history.history

plt.plot(hist["accuracy"], label="accuracy")
plt.plot(hist["loss"], label="loss")

if "val_accuracy" in hist and "val_loss" in hist:
    plt.plot(hist["val_accuracy"], label="val_accuracy")
    plt.plot(hist["val_loss"], label="val_loss")

plt.ylabel("Accuracy / Loss")
plt.xlabel("Epochs #")
plt.legend()

plt.show()

In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("Mobile_Net_trained.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved successfully!")

Saved artifact at '/tmp/tmpuool_62i'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 55), dtype=tf.float32, name=None)
Captures:
  138199142323152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138199142322384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138199142325264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138199142324880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138199142323728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138199142325456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138199142324496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138199142322768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138199142325648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138199142323344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138199142